In [1]:
import cv2
import torch
import serial
import time
import math
from IPython.display import display, clear_output
from PIL import Image


model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5s_best.pt', source='github')

Using cache found in C:\Users\Dell/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-4-14 Python-3.10.10 torch-2.6.0+cpu CPU

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [4]:

model.conf = 0.3


serialcomm = serial.Serial('COM3', 9600)
serialcomm.timeout = 1


cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)

CENTER_X = 640
CENTER_Y = 360
A, B, C = 0.0126, -2.7105, 182.62

class Turret:
    def __init__(self, obj_x, obj_y, obj_z):
        self.obj_x = obj_x
        self.obj_y = obj_y
        self.obj_z = obj_z

    def offsets(self, offs_x=0, offs_y=0, offs_z=0):
        self.offs_x = offs_x
        self.offs_y = offs_y
        self.offs_z = offs_z

    def getAngles(self):
        z = self.obj_z - self.offs_z
        x = abs(self.obj_x) + self.offs_x if self.obj_x > 0 else abs(self.obj_x) - self.offs_x
        y = abs(self.obj_y) - self.offs_y if self.obj_y > 0 else abs(self.obj_y) + self.offs_y

        theta_x = math.degrees(math.atan(x / z))
        theta_x = 90 + theta_x if self.obj_x > 0 else 90 - theta_x

        theta_y = math.degrees(math.atan(y / z))
        theta_y = 90 - theta_y if self.obj_y > 0 else 90 + theta_y

        return int(theta_x), int(theta_y)


fire_detected_start = None
fire_confirmed = False
FIRE_DETECT_DURATION = 1 
last_fire_time = 0


last_signal_time = 0
SIGNAL_INTERVAL = 0  

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        fire_detected = False
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = model(rgb_frame)
        detections = results.xywh[0].cpu().numpy()

        if len(detections) > 0:
            x, y, w, h, conf = detections[0][:5]
            if conf > 0.2:
                fire_detected = True

                if fire_detected_start is None:
                    fire_detected_start = time.time()
                elif time.time() - fire_detected_start >= FIRE_DETECT_DURATION:
                    fire_confirmed = True
                    last_fire_time = time.time()

                x1, y1 = int(x - w / 2), int(y - h / 2)
                x2, y2 = int(x + w / 2), int(y + h / 2)

                center_x = (x1 + x2) // 2
                center_y = (y1 + y2) // 2
                width = x2 - x1

                distance_virtual = width
                Z_real = A * distance_virtual**2 + B * distance_virtual + C

                X_virtual = -(center_x - CENTER_X)
                Y_virtual = -(center_y - CENTER_Y)
                X_real = X_virtual * (6.3 / distance_virtual)
                Y_real = Y_virtual * (6.3 / distance_virtual)

                turret = Turret(X_real, Y_real, Z_real)
                turret.offsets(12, 0, 7)
                theta_x, theta_y = turret.getAngles()

                serialcomm.write(f"%X{theta_x + 5}#".encode())
                serialcomm.write(f"%Y{theta_y - 3}#".encode())

                conf_percent = round(conf * 100, 1)
                label = f"FIRE: {conf_percent}%"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        else:
            fire_detected_start = None
            fire_confirmed = False

        current_time = time.time()
        if fire_confirmed:
            if current_time - last_signal_time >= SIGNAL_INTERVAL:
                serialcomm.write(b"%RELAY#")
                last_signal_time = current_time
        else:
            serialcomm.write(b"%OFF#")

        clear_output(wait=True)
        display(Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))
        time.sleep(0.1)

except KeyboardInterrupt:
    pass
finally:
    cap.release()
    serialcomm.close()
    print("System stopped.")


System stopped.
